In [1]:
import pandas as pd
from db.helpers import new_sales_collection

In [2]:

df = pd.DataFrame(new_sales_collection.find())
area_df = pd.read_excel("./seasonalities.xlsx", "area")
industry_df = pd.read_excel("./seasonalities.xlsx", "industry")
location_type_df = pd.read_excel("./seasonalities.xlsx", "location_type")
product_focus_df = pd.read_excel("./seasonalities.xlsx", "product_focus")

df = df.merge(
    area_df,
    how="left",
    on=["Level_3_Area", "Sales_Year", "Sales_Month"],
    suffixes=("", "_area"),
)
df = df.merge(
    industry_df,
    how="left",
    on=["Industry_Level_2", "Sales_Year", "Sales_Month"],
    suffixes=("", "_industry"),
)
df = df.merge(
    location_type_df,
    how="left",
    on=["Location_Type", "Sales_Year", "Sales_Month"],
    suffixes=("", "_location_type"),
)
df = df.merge(
    product_focus_df,
    how="left",
    on=["Product_Focus", "Sales_Year", "Sales_Month"],
    suffixes=("", "_product_focus"),
)

In [3]:
"""
Seasonalities
"""
df["weekday_store_sales_seasonality"] = (
    df.loc[:, "Weekday_Store_Sales_area"]
    + df.loc[:, "Weekday_Store_Sales_industry"]
    + df.loc[:, "Weekday_Store_Sales_product_focus"]
    + df.loc[:, "Weekday_Store_Sales_location_type"]
) / 4
df["weekday_delivery_sales_seasonality"] = (
    df.loc[:, "Weekday_Delivery_Sales_area"]
    + df.loc[:, "Weekday_Delivery_Sales_industry"]
    + df.loc[:, "Weekday_Delivery_Sales_product_focus"]
    + df.loc[:, "Weekday_Delivery_Sales_location_type"]
) / 4
df["weekend_store_sales_seasonality"] = (
    df.loc[:, "Weekend_Store_Sales_area"]
    + df.loc[:, "Weekend_Store_Sales_industry"]
    + df.loc[:, "Weekend_Store_Sales_product_focus"]
    + df.loc[:, "Weekend_Store_Sales_location_type"]
) / 4
df["weekend_delivery_sales_seasonality"] = (
    df.loc[:, "Weekend_Delivery_Sales_area"]
    + df.loc[:, "Weekend_Delivery_Sales_industry"]
    + df.loc[:, "Weekend_Delivery_Sales_product_focus"]
    + df.loc[:, "Weekend_Delivery_Sales_location_type"]
) / 4

In [4]:
df.drop(
    [
        "Weekday_Store_Sales_area",
        "Weekday_Delivery_Sales_area",
        "Weekend_Store_Sales_area",
        "Weekend_Delivery_Sales_area",
        #
        "Weekday_Store_Sales_industry",
        "Weekday_Delivery_Sales_industry",
        "Weekend_Store_Sales_industry",
        "Weekend_Delivery_Sales_industry",
        #
        "Weekday_Store_Sales_location_type",
        "Weekday_Delivery_Sales_location_type",
        "Weekend_Store_Sales_location_type",
        "Weekend_Delivery_Sales_location_type",
        #
        "Weekday_Store_Sales_product_focus",
        "Weekday_Delivery_Sales_product_focus",
        "Weekend_Store_Sales_product_focus",
        "Weekend_Delivery_Sales_product_focus",
    ],
    axis=1,
    inplace=True,
)

In [5]:
def f(df: pd.DataFrame):
    prevRow = {
        "Weekday_Store_Sales": pd.NA,
        "Weekday_Delivery_Sales": pd.NA,
        "Weekend_Delivery_Sales": pd.NA,
        "Weekend_Store_Sales": pd.NA,
    }
    for i, row in df.iterrows():
        if i == 0:
            prevRow = {
                "Weekday_Store_Sales": pd.NA,
                "Weekday_Delivery_Sales": pd.NA,
                "Weekend_Delivery_Sales": pd.NA,
                "Weekend_Store_Sales": pd.NA,
            }
            continue
        if (
            pd.isna(row.Monthly_Sales)
            and row.Source == "Generated"
            and pd.notna(prevRow["Weekday_Store_Sales"])
        ):
            weekday_delivery_sales_seasonality = (
                row.weekday_delivery_sales_seasonality
            )
            weekday_store_sales_seasonality = row.weekday_store_sales_seasonality
            weekend_store_sales_seasonality = row.weekend_store_sales_seasonality
            weekend_delivery_sales_seasonality = (
                row.weekend_delivery_sales_seasonality
            )
            df.at[i, "Weekday_Store_Sales"] = prevRow["Weekday_Store_Sales"] + (
                prevRow["Weekday_Store_Sales"] * weekday_store_sales_seasonality
            )
            df.at[i, "Weekday_Delivery_Sales"] = prevRow[
                "Weekday_Delivery_Sales"
            ] + (
                prevRow["Weekday_Delivery_Sales"]
                * weekday_delivery_sales_seasonality
            )
            df.at[i, "Weekend_Delivery_Sales"] = prevRow[
                "Weekend_Delivery_Sales"
            ] + (
                prevRow["Weekend_Delivery_Sales"]
                * weekend_delivery_sales_seasonality
            )
            df.at[i, "Weekend_Store_Sales"] = prevRow["Weekend_Store_Sales"] + (
                prevRow["Weekend_Store_Sales"] * weekend_store_sales_seasonality
            )
        prevRow["Weekday_Store_Sales"] = df.at[i, "Weekday_Store_Sales"]
        prevRow["Weekday_Delivery_Sales"] = df.at[i, "Weekday_Delivery_Sales"]
        prevRow["Weekend_Delivery_Sales"] = df.at[i, "Weekend_Delivery_Sales"]
        prevRow["Weekend_Store_Sales"] = df.at[i, "Weekend_Store_Sales"]
    return df

In [6]:
print(len(df[(df["Weekday_Store_Sales"].isna())]))
df = df.groupby("Reference_Full_ID").apply(f)
print(len(df[df["Weekday_Store_Sales"].isna()]))

347457
220824


In [8]:
display(df)

_id  Primary_ID Primary_Sheet  \
Reference_Full_ID                                                              
Foodservice 1     0       64de1fc166c496bec71a215d           1         Sales   
                  1       64de1fc166c496bec71a215e           2         Sales   
                  2       64de1fc166c496bec71a215f           3         Sales   
                  3       64de1fc166c496bec71a2160           4         Sales   
                  4       64de1fc166c496bec71a2161           5         Sales   
...                                            ...         ...           ...   
Service 9830      162791  65a8e1f150a86658d34b7a01      212039         Sales   
                  162792  65a8e1f150a86658d34b7a02      212040         Sales   
                  162793  65a8e1f150a86658d34b7a03      212041         Sales   
                  162794  65a8e1f150a86658d34b7a04      212042         Sales   
                  162795  65a8e1f150a86658d34b7a05      212043         Sales   

                         Primary_Full_ID  Reference_ID Reference_Sheet  \
Reference_Full_ID                                                        
Foodservice 1     0              Sales 1             1     Foodservice   
                  1              Sales 2             1     Foodservice   
                  2              Sales 3             1     Foodservice   
                  3              Sales 4             1     Foodservice   
                  4              Sales 5             1     Foodservice   
...                                  ...           ...             ...   
Service 9830      162791    Sales 212039          9830         Service   
                  162792    Sales 212040          9830         Service   
                  162793    Sales 212041          9830         Service   
                  162794    Sales 212042          9830         Service   
                  162795    Sales 212043          9830         Service   

                         Reference_Full_ID                    Company_Name  \
Reference_Full_ID                                                            
Foodservice 1     0          Foodservice 1             Coffee Shop Company   
                  1          Foodservice 1             Coffee Shop Company   
                  2          Foodservice 1             Coffee Shop Company   
                  3          Foodservice 1             Coffee Shop Company   
                  4          Foodservice 1             Coffee Shop Company   
...                                    ...                             ...   
Service 9830      162791      Service 9830  Kuwait National Cinema Company   
                  162792      Service 9830  Kuwait National Cinema Company   
                  162793      Service 9830  Kuwait National Cinema Company   
                  162794      Service 9830  Kuwait National Cinema Company   
                  162795      Service 9830  Kuwait National Cinema Company   

                         Industry_Level_2 Product_Focus  ...     Source  \
Reference_Full_ID                                        ...              
Foodservice 1     0            Coffeeshop        Coffee  ...   Estimate   
                  1            Coffeeshop        Coffee  ...   Estimate   
                  2            Coffeeshop        Coffee  ...   Estimate   
                  3            Coffeeshop        Coffee  ...   Estimate   
                  4            Coffeeshop        Coffee  ...   Estimate   
...                                   ...           ...  ...        ...   
Service 9830      162791    Entertainment        Cinema  ...  Generated   
                  162792    Entertainment        Cinema  ...  Generated   
                  162793    Entertainment        Cinema  ...  Generated   
                  162794    Entertainment        Cinema  ...  Generated   
                  162795    Entertainment        Cinema  ...  Generated   

                              Study    Researcher Sa